In [22]:
import pandas as pd
import numpy as np
import nltk
from collections import Counter #uso para n-gramas
import re 
import plotly.express as px #biblio q pus pra essa viz
from sklearn.manifold import TSNE #biblio q pus pra essa viz

In [23]:
csv_file_path = 'C:/Users/Administrator/Documents/repos/mestrado/visual_analytics/corpus_completo.csv'

# Lendo o csv como um df
df = pd.read_csv(csv_file_path)

#Criando uma cópia
df_va = df.copy()

In [24]:
df_va.head()

,file_name,content,nota
0,20152t4p1166n0r.docx,É inaceitável e constrangedor a violência que ...,0
1,20152t4p1305n0r.docx,A capital paraense já foi considerada uma das ...,0
2,20152t4p1477n0r.docx,A prefeitura municipal respeitosa. ...,0
3,20152t4p1506n0r.docx,Data: 20/10/2015\nDe: Bibiana C. Terra\nPara: ...,0
4,20152t4p1512n0r.docx,Desculpe. Eu não conheco-me a matéria “Azulejo...,0


In [25]:
df_va.groupby('nota').count()

,file_name,content
nota,,
0,25,25
1,211,211
2,628,628
3,715,715
4,477,477
5,237,237


In [26]:
nltk.download('stopwords')
nltk.download('punkt') # é um tokenizador, importante para nltk.word_tokenize
# nltk.download('rslp') é um stemmer. acho q nao vou usar
nltk.download('punkt_tab') # Precisei que devia fazer download desse pacote pq o punkt nao tava funcionando

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Administrator\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Administrator\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\Administrator\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [27]:
# Pré-processamento

# Devo tirar acentos?? Ex: Belém
# Retirar números? Na duvida, retirei.
# Não fiz stemming visto que poderia perder erros ortográficos

#Retirada de sinais gráficos, pontuações e espaços
def clean_cp(text):
    cleaned = text.lower() #Deixando tudo minúsculo
    cleaned = re.sub('[^\w\s]', '', cleaned) # Removendo pontuacao
    #cleaned = re.sub('[0-9]+', '', cleaned) # Removendo números 
    cleaned = re.sub('\d+', '', cleaned) # Removendo números NÃO TAVA FUNCIONANDO. Começou a funcionar qdo pus o lower como primeiro comando da funçao
    cleaned = re.sub('\s+', ' ', cleaned) # Removendo espaços extras
    cleaned = re.sub('\s+', ' ', cleaned)
    return cleaned.strip() # Removendo tabs

df_va['content'] = df_va['content'].apply(clean_cp)
#df_va['content'] = df_va['content'].apply(stopwords_cp)

In [28]:
df_va['content'].sample(5)

187     estimados prefeitos municipais como morador de...
1912    data de bibiana arminda cambará para prefeitur...
1948    belém de outubro de à prefeitura municipal de ...
2211    prezados prefeito vereadores e governantes est...
124     universidade de xxx prefeitura municipal depor...
Name: content, dtype: object

In [29]:
df_va.dtypes

file_name    object
content      object
nota          int64
dtype: object

In [30]:
# Tokenizando e retirando stopwords: retiro stopwords pq embeddings fixas nao consideram contexto

def tokenized_cp(text):
   stopwords = nltk.corpus.stopwords.words('portuguese') # Carregando as stopwords do português
   tokenized = nltk.word_tokenize(text, language='portuguese') #Transforma o texto em tokens
   text_sem_stopwords = [token for token in tokenized if token not in stopwords] # Deixando somente o que nao é stopword no texto
   return text_sem_stopwords

df_va['tokenized_content'] = df_va['content'].apply(tokenized_cp)

In [31]:
df_va['tokenized_content'].sample(5)

1230    [belém, outubro, doutor, prefeito, municipal, ...
1843    [prefeitura, municipal, bibiana, cambará, mora...
1217    [belém, cidade, belas, importantes, cidades, p...
278     [arequipa, octobro, senhor, prefeito, agrado, ...
1715    [julho, prefeitura, municipal, belém, referênc...
Name: tokenized_content, dtype: object

In [32]:
from gensim.models import Word2Vec
import streamlit as st

In [12]:
df_va['tokenized_content']

0       [inaceitável, constrangedor, violência, existe...
1       [capital, paraense, considerada, cidades, bras...
2       [prefeitura, municipal, respeitosa, publicidad...
3       [data, bibiana, c, terra, prefeitura, municipa...
4       [desculpe, conhecome, matéria, azulejos, valio...
                              ...                        
2288    [belem, outubro, senhores, prefeitura, municip...
2289    [pará, outubro, senhor, prefeito, municipal, p...
2290    [pilas, outubro, prefeitura, municipal, belém,...
2291    [prefeitura, municipal, morador, belém, senhor...
2292    [cidade, belém, dia, outubro, exelentíssima, p...
Name: tokenized_content, Length: 2293, dtype: object

In [17]:
# Ensure the 'conteudo' column is properly tokenized
#df_va['tokenized_content'] = df_va['tokenized_content'].apply(eval)  # Convert string tokens to lists if stored as strings ARRUMAR

In [33]:
# Prepare the tokenized corpus
tokenized_corpus = df_va['tokenized_content'].tolist()

In [34]:
tokenized_corpus

[['inaceitável',
  'constrangedor',
  'violência',
  'existe',
  'brasil',
  'causa',
  'pânico',
  'acomete',
  'vidas',
  'inocentes',
  'ter',
  'vida',
  'cheia',
  'harmônia',
  'tranquilidade',
  'podemos',
  'dar',
  'solução',
  'ounque',
  'acredito',
  'todas',
  'esferas',
  'governo',
  'segurança',
  'tomando',
  'todas',
  'medidas',
  'necessárias',
  'convater',
  'violência',
  'país',
  'acho',
  'dar',
  'certo',
  'disminuição',
  'violência',
  'tarefa',
  'fácil',
  'dar',
  'olhada',
  'muitos',
  'aspectos',
  'medidas',
  'ter',
  'acesso',
  'educação',
  'abolir',
  'comerciais',
  'mostram',
  'tv',
  'erradicar',
  'venda',
  'drogas',
  'cigarros',
  'bebidas',
  'alcoioólicas',
  'outras',
  'coisas'],
 ['capital',
  'paraense',
  'considerada',
  'cidades',
  'brasileiras',
  'maior',
  'variedade',
  'azulejos',
  'coloriam',
  'fachadas',
  'enterior',
  'residências',
  'boa',
  'parte',
  'importadas',
  'europa',
  'década',
  'cá',
  'azulejos',
  

Do Lab 4 do prof Dennis:

Alguns parametros do Word2vec

vector_size – dimensionalidade dos vetores das palavras.

window – tamanho do contexto a considerar, por exemplo windows=5 irá considerar as 5 palavras à esquerda e as 5 palavras à direita da palavra atual como a janela de contexto. O modelo tentará então prever a palavra atual dado este contexto.

min_count – ignora palavras com frequência total menor do que min_count.

sg – o algoritmo de treinamento: 1 for skip-gram e diferente disto CBOW.

DEVO REMOVER STOPWORDS????

Ver: [artigo_geekforgeeks](https://www.geeksforgeeks.org/word2vec-with-gensim/)

In [35]:
# Train the Word2Vec model
model = Word2Vec(
    sentences=tokenized_corpus,  # Input tokenized sentences
    vector_size=100,            # Dimensionality of the embedding vectors
    window=5,                   # Context window size
    min_count=1,                # Minimum word frequency
    sg=0,                       # CBOW (0) or Skip-gram (1)
    workers=4                   # Number of worker threads
)

# Parâmetros que prof usou: model = gensim.models.Word2Vec(sentences=sentences, vector_size=100, window=5, min_count=1, epochs=20, sg=1)
# Em resumo, ele não usou workers mas usou epochs=20... No artigo linkado acima tá com ms mesmos parametros do chatgpt.. o q faz sentido ja q puxa do google.. pensar certinho a respeito

In [36]:
# Save the trained model          COMO A PRINCIPIO VOU RODAR NA HORA, NAO CHEGUEI A SALVAR
model_path = "fixed_word2vec_model.model"
model.save(model_path)
print(f"Model saved to {model_path}")

Model saved to fixed_word2vec_model.model


In [ ]:
# Streamlit app example for loading and using the model
#st.title("Fixed Word Embedding with Word2Vec") ### PEDI AJUDA PRA CRIAR ALGO Q FICASSE PRE-SALVO PRA NAO RODAR A CADA VEZ

In [ ]:
# A Streamlit app loads the saved model with @st.cache_resource to avoid reloading it multiple times.
# Users can input a word to retrieve its embedding.


#@st.cache_resource    ### MENCIONEI Q GOSTARIA DE MOSTRAR O EMBEDDING NO STREAMLIT E VEJO ESSE PEDAÇO COMO UM EXTRA
#def load_model():
 #   return Word2Vec.load(model_path)

#word2vec_model = load_model()

#word = st.text_input("Enter a word to get its embedding:")
#if word:
   # if word in word2vec_model.wv:
    #    embedding = word2vec_model.wv[word]
     #   st.write("Embedding:", embedding)
   # else:
    #    st.write(f"The word '{word}' is not in the vocabulary.")

To visualize the embeddings in a Jupyter Notebook, you can use techniques like dimensionality reduction (e.g., PCA or t-SNE) to project the high-dimensional embeddings into 2D or 3D space. Here's how you can do it:

Steps to Visualize Word Embeddings

Load the Trained Model: Use the saved Word2Vec model to access word vectors.

Extract Embeddings: Retrieve the vectors and corresponding words from the model.

Dimensionality Reduction: Use PCA or t-SNE to reduce the dimensionality of the embeddings to 2D for visualization.

Plot the Embeddings: Use libraries like matplotlib or plotly for plotting.

Key Points:

Dimensionality Reduction:

PCA is faster but may not capture non-linear relationships as well.

t-SNE is better for visualizing clusters but slower for larger datasets.

Annotations:

To avoid overcrowding the plot, only annotate a subset of words (e.g., the top 50 most frequent words).

Optional Exploration:

Use plotly for interactive 2D/3D plots if you'd like dynamic visualization.

In [51]:
# TSNE visualization function

def visualize_embeddings(model, num_points=1000):
    words = list(model.wv.index_to_key)[:num_points]  # num_points limits the number of words to display
    vectors = model.wv[words]

    tsne = TSNE(n_components=2, random_state=42, perplexity=30)
    reduced_vectors = tsne.fit_transform(vectors)

    df_fe = pd.DataFrame({
        'Token': words,
        'x': reduced_vectors[:, 0],
        'y': reduced_vectors[:, 1]
    })

    fig = px.scatter(
        df_fe, x='x', y='y', text='Token',
        title="t-SNE Visualization of Word Embeddings",
        labels={'x': 'Dimension 1', 'y': 'Dimension 2'}
    )
    fig.update_traces(
        textposition='top center',
        marker=dict(color='#FF4B4B'),  # Streamlit red color
        textfont=dict(color='white')  # White words
    )
    fig.update_layout(
        plot_bgcolor='rgba(0, 0, 0, 0)',  # Transparent plot background
        paper_bgcolor='rgba(0, 0, 0, 0)',  # Transparent outer background
        font=dict(color='white'),  # White font for axis labels and title
        hoverlabel=dict(
            bgcolor='white',  # White background for tooltips
            font_size=12,     # Tooltip font size
            font_color='black'  # Tooltip font color
        )
    )
    return fig

In [52]:
visualize_embeddings(model)

In [ ]:
# st.plotly_chart(visualize_embeddings(model)) pra exibir no streamlit

The visualization is limited by the parameter num_points, which controls the number of words used in the visualization. Let’s break it down step by step:

Function Signature:

def visualize_embeddings(model, num_points=100):

The function takes model (the trained Word2Vec model) and num_points (the maximum number of words to visualize).

Selecting Words:

words = list(model.wv.index_to_key)[:num_points]

model.wv.index_to_key contains the list of words in the vocabulary, sorted by their frequency (most frequent words appear first).

The [:num_points] slices the list to select only the first num_points words.

Limiting the Vectors:

vectors = model.wv[words]

This retrieves the corresponding word vectors for the selected num_points words.


Dynamic Limit:

In your Streamlit app, the number of points to visualize is controlled by the slider:

num_points = st.slider("Number of words to visualize", min_value=10, max_value=200, value=50)

The default value is 50 (value=50), and the user can adjust it between 10 and 200.


To Summarize:

num_points dynamically limits how many words are used for visualization.

The list model.wv.index_to_key ensures that only the top num_points most frequent words are selected.

In [ ]:
# Add visualization to Streamlit app
#if st.checkbox("Visualize Embeddings"):
  #  num_points = st.slider("Number of words to visualize", min_value=10, max_value=200, value=50)
   # fig = visualize_embeddings(word2vec_model, num_points=num_points)
   # st.plotly_chart(fig)

#word = st.text_input("Enter a word to get its embedding:")
#if word:
 #   if word in word2vec_model.wv:
    #    embedding = word2vec_model.wv[word]
    #    st.write("Embedding:", embedding)
   # else:
    #    st.write(f"The word '{word}' is not in the vocabulary.")

In [ ]:
# Pendencias:

# FODAC FODAC dependendo, fazer para cada grupo de notas... ou mostrar infos de cada palavra... quem sabe com o percentual q aparece em cada texto. 'a palavra x aparece em xx% dos textos de nota 5, '